Init:

In [1]:
import numpy as np
from numpy import pi,sqrt,sin,cos,tan,arcsin,arccos,arctan
from numpy import real as re
from numpy import imag as im
from numpy import conj as conj
from numpy import exp  as exp

import sympy as sp
from sympy import limit, Symbol, oo

import matplotlib.pyplot as plt

from IPython.display import display,Math,HTML,Latex # Used to display widgets in the notebook
sp.init_printing(use_unicode=True,use_latex=True)

import ipywidgets as widgets #If not install run <pip install ipywidgets> in DOS command prompt (Anaconda distribution)

import time

print('Init done:', time.ctime() )

%matplotlib inline 
#%connect_info

Init done: Fri Feb 19 14:38:17 2016


Original Kundur formulas

In [2]:
#Symbolic variablde definitions http://docs.sympy.org/dev/modules/core.html
EsSym,ZlnSym,ZldTsym,VrSym,ZsSym,ZrSym,Fsym=sp.symbols( r"E_s,Z_{ln},Z_{ldT},V_r,Z_{s},Z_{r},F",positive=True)
thetaSym,phiSym=sp.symbols( r"\theta,\varphi",real=True)
ZsrRatioSym=sp.symbols( r"Z_{srRt}",positive=True)

PrEq =ZrSym/Fsym *(EsSym/ZsSym)**2*sp.cos(phiSym)
Ieq  =1/sp.sqrt(Fsym) * EsSym/ZsSym
UrEq =1/sp.sqrt(Fsym) * ZrSym/ZsSym *EsSym
Feq =1+(ZrSym/ZsSym)**2 +2*(ZrSym/ZsSym)*sp.cos(thetaSym-phiSym)

display(Math(r' P_r= %s' %sp.latex(PrEq)  ))
display(Math(r' I= %s' %sp.latex(Ieq)  ))
display(Math(r' U_r= %s' %sp.latex(UrEq)  ))
display(Math(r' F= %s' %sp.latex(Feq)  ))
display(Math(r' \theta =\angle \underline{Z}_{s} ' ))
display(Math(r' \phi   =\angle \underline{Z}_{r} ' ))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Insertion of F

In [3]:

PrEq   = PrEq.subs(Fsym,Feq)
Ieq    =  Ieq.subs(Fsym,Feq)
UrEq   = UrEq.subs(Fsym,Feq)

PrEq   = PrEq.subs(ZrSym/ZsSym,1/ZsrRatioSym)
Ieq    =  Ieq.subs(ZrSym/ZsSym,1/ZsrRatioSym)
UrEq   = UrEq.subs(ZrSym/ZsSym,1/ZsrRatioSym)


PrDiffEq = sp.diff(PrEq,ZsrRatioSym)
sol =sp.solve(PrDiffEq,ZsrRatioSym)[0] #One solution (ZsrRatioSym=1)

PrCrEq     = PrEq.subs(ZsrRatioSym,sol)
PrCrMaxEq = PrCrEq.subs([(phiSym,0),(thetaSym,pi/2)])
#Nominator max: cos(phi)=1 -> phi =0° 
#Denominater min: cos(theta-phi)=0 -> theta=90°+phi=90°

ImaxEq = limit(Ieq, ZsrRatioSym,oo )

display(Math(r' P_r= %s' %sp.latex(PrEq)  ))
display(Math(r' I= %s' %sp.latex(Ieq)  ))
display(Math(r' U= %s' %sp.latex(UrEq)  ))
display(Math(r'Z_{rt,cr}= \left(\frac{Z_s}{Z_r}\right)_{cr} = %s' %sp.latex(sol)  ))
display(Math(r' P_{r,cr}= %s' %sp.latex(PrCrEq)  ))
display(Math(r' P_{r,cr,max}= %s' %sp.latex(PrCrMaxEq)  ))
display(Math(r' I_{max}= %s' %sp.latex(ImaxEq)  ))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Insertion of numbers:

In [4]:
##### #Data:
SnTf = 31.5  #Nominal trafo power [MVA]
UnTfHv = 110 #Nominal trafo high voltage[kv]                       
UnTfLv = 20  #Nominal trafo low voltage (medium voltage) [kv]
ukx   = 12.8 /100 # [%]
ukr   = 0.37 /100 # [%]
Skss = 100 #Short circuit power at high voltage level [MVA]

#Grid:
Ugr =110 #[kV]
Xgr= Ugr**2/Skss
Rgr=0
ZgrCpx= Rgr +1j* Xgr

#Line(s):
ZlnCpx = (0.2+1j*0.4) * 100 #[Ohm] 
#ZlnCpx=ZlnCpx/2 #Two lines in operation
#ZlnCpx = 1j*20

#Trafo:
Xtf= ukx * UnTfHv**2/SnTf #Transformer impedance on high voltage level
Rtf= ukr * UnTfHv**2/SnTf #Transformer resistance on high voltage level
ZtfCpx =Rtf+1j*Xtf
a =UnTfHv/UnTfLv

#Load:
#cosPhi=0.95
#Pld= 1 #[MW]
#SldCpx = Pld /cosPhi * exp(1j*arccos(cosPhi))  #[MVA]
#SldCpx=1+1j*0.5
#UnLvCpx =UnLv
#ZldCpx =conj( abs(UnLvCpx)**2/SldCpx )  
ZldCpx=324
Rld =re(ZldCpx)
Xld=0
RldT = Rld * a**2
XldT = Xld * a**2
ZldTcpx =RldT +1j*XldT

#Calculate sending and receiving impedances Xs and Xr:
ZsCpx  = ZgrCpx + ZlnCpx + ZtfCpx 
Zs= abs(ZsCpx)
Rs =re(ZsCpx)
Rs =im(ZsCpx)
theta=np.angle(ZsCpx)

ZrCpx  = ZldTcpx 
Zr= abs(ZrCpx)
Rs =re(ZrCpx)
Rs =im(ZrCpx)
phi=np.angle(ZrCpx)

#Critical I and P calculation:
Imax    =float(ImaxEq.subs([(EsSym,Ugr/sqrt(3)),(ZsSym,Zs)]))
PrCr    =float(PrCrEq.subs([(EsSym,Ugr),(ZsSym,Zs),(thetaSym,theta),(phiSym,phi)]) )
PrCrMax =float(PrCrMaxEq.subs([(EsSym,Ugr),(ZsSym,Zs)]) )

#Display:
display(Math(r' \underline{Z}_{gr}   = %s' %sp.latex("%.4f+%.4fj" %(re(ZgrCpx),im(ZgrCpx))) ))
display(Math(r' \underline{Z}_{ln}  = %s' %sp.latex("%.4f+%.4fj" %(re(ZlnCpx),im(ZlnCpx))) ))
display(Math(r' \underline{Z}_{tf,HV}= %s' %sp.latex("%.4f+%.4fj" %(re(ZtfCpx),im(ZtfCpx))) )) #round on 4. digit?
display(Math(r' \underline{Z}_{s}= %s' %sp.latex("%.4f+%.4fj" %(re(ZsCpx),im(ZsCpx))) ))
#display(Math(r' \underline{Z}_{ld}= %s'    %sp.latex("%.4f+%.4fj" %(re(ZldCpx),im(ZldCpx))) ))
#display(Math(r' \underline{Z}_{ldT}= %s'   %sp.latex("%.4f+%.4fj" %(re(ZldTcpx),im(ZldTcpx))) ))
display(Math(r' I_{max,Hv}= %s kA' %sp.latex("%.4f" %Imax)  ))
display(Math(r' I_{max,Lv}= %s kA' %sp.latex("%.4f" %(Imax*a))  ))
display(Math(r' P_{cr}= %s MW' %sp.latex("%.4f" % PrCr)  ))
display(Math(r' P_{cr,max}= %s MW' %sp.latex("%.4f" % PrCrMax)  ))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>